<a href="https://colab.research.google.com/github/SPORTNERHSE/SPORTNER/blob/master/vector_models_English_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import gensim
import pandas as pd
import re
from tqdm import tqdm
import numpy as np
import nltk
from multiprocessing import cpu_count

In [0]:
from string import punctuation

In [0]:
punct = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~»—«–'

In [0]:
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')
stops = stopwords.words('russian') + list(punctuation) + ['»', '—', '«', '–']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
df = pd.read_csv('result.tar.gz', compression='gzip', sep='\t', error_bad_lines=False)

b'Skipping line 37039: expected 12 fields, saw 15\nSkipping line 45611: expected 12 fields, saw 15\n'
b'Skipping line 354339: expected 12 fields, saw 18\nSkipping line 360804: expected 12 fields, saw 102\nSkipping line 360819: expected 12 fields, saw 78\nSkipping line 379126: expected 12 fields, saw 14\nSkipping line 444446: expected 12 fields, saw 16\nSkipping line 449079: expected 12 fields, saw 16\nSkipping line 450250: expected 12 fields, saw 15\n'
b'Skipping line 481374: expected 12 fields, saw 18\nSkipping line 510619: expected 12 fields, saw 20\nSkipping line 512271: expected 12 fields, saw 14\nSkipping line 514317: expected 12 fields, saw 15\nSkipping line 519707: expected 12 fields, saw 40\n'
b'Skipping line 538495: expected 12 fields, saw 13\n'
b'Skipping line 765901: expected 12 fields, saw 56\nSkipping line 773310: expected 12 fields, saw 43\n'
b'Skipping line 805899: expected 12 fields, saw 13\nSkipping line 868492: expected 12 fields, saw 21\nSkipping line 871801: expecte

In [0]:
teams_names_in_russian_and_english_df = pd.read_csv("teams names in russian and english.csv")
leagues_names_in_russian_and_english_df = pd.read_csv("leagues names in russian and english.csv")

In [0]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)

In [0]:
df = df.replace('', np.NaN)
df = df.replace('\xa0', np.NaN)
df = df.dropna()
df =  df[df['result.tsv'].str.contains('[A-Za-z]')]
df = df[df['result.tsv'].str.split().str.len().gt(1)]
df = df.dropna()
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [0]:
def which_language (text):
    if bool(re.search('[\u0400-\u04FF]', text)) == True:
        return 'russian'
    else:
        return 'english'

In [0]:
lang=[]
for i,text in enumerate(df['result.tsv'].values): 
    lang.append(which_language(str(text)))

In [0]:
df['Language'] = lang
grouped=df.groupby('Language')
for name, group in grouped: 
    if name == 'english': 
        english = group.drop('Language',axis=1) 
    else: 
        russian = group.drop('Language',axis=1)

In [0]:
english.shape

(89138, 8)

In [0]:
english = pd.merge(english, teams_names_in_russian_and_english_df,  how='left', left_on=['team_of_season_1_name_in_russian'], right_on = ['team name in Russian'])
english.rename({'English': 'team_of_season_1_name_in_english'}, axis=1, inplace=True)

english = pd.merge(english, teams_names_in_russian_and_english_df,  how='left', left_on=['team_of_season_2_name_in_russian'], right_on = ['team name in Russian'])
english.rename({'English': 'team_of_season_2_name_in_english'}, axis=1, inplace=True)

english = pd.merge(english, leagues_names_in_russian_and_english_df,  how='left', left_on=['league_name_in_russian'], right_on = ['leagues name in Russian'])
english.rename({'unique_leagues_in_english': 'league_name_in_english'}, axis=1, inplace=True)


In [0]:
english.drop(['team name in Russian_x' , 'team name in Russian_y' , 'leagues name in Russian' ] , axis=1 , inplace=True)

In [0]:
# Here we are just re-naming the columns so one check that what's in it and interpret the information easily 
english.rename({'team name in English_x': 'team_of_season_1_name_in_english'}, axis=1, inplace=True)
english.rename({'team name in English_y': 'team_of_season_2_name_in_english'}, axis=1, inplace=True)
english.rename({'leagues names in English': 'league_name_in_english'}, axis=1, inplace=True)

In [0]:
# Here we are just setting the correct data type that english names should be string
english['team_of_season_1_name_in_english']=english['team_of_season_1_name_in_english'].astype(str)
english['team_of_season_2_name_in_english']=english['team_of_season_2_name_in_english'].astype(str)
english['league_name_in_english']=english['league_name_in_english'].astype(str)

In [0]:
english.head(3)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,team_of_season_1_name_in_english,team_of_season_2_name_in_english,league_name_in_english
0,Middlesbrough have woken to good times in thei...,7,0-0,bet365,2019-04-19 14:00:00,Чемпионшип,Мидлсбро,Сток Сити,Middlesbrough,Stoke City,Championship
1,Wolves prefer to attack with pace on the count...,7.5,0-0,bet365,2019-04-20 14:00:00,Премьер-лига,Вулверхэмптон,Брайтон,Wolverhampton,Brighton,Premier League
2,In my opinion a 0-0 draw is quite likely betwe...,13,0-0,bet365,2019-04-18 19:00:00,Лига Европы УЕФА,Валенсия,Вильярреал,Valencia,Villarreal,UEFA Europa League


In [0]:
teams1_match = np.zeros(english.shape[0])
for i, team1 in enumerate(english['team_of_season_1_name_in_english']):
    if team1 in english['result.tsv'].values[i] :
        teams1_match[i] = 1 
    else: 
        teams1_match[i] = 0
english['direct_match_for_teams1'] = teams1_match

In [0]:
teams2_match = np.zeros(english.shape[0])
for i, team2 in enumerate(english['team_of_season_2_name_in_english']):
    if team2 in english['result.tsv'].values[i] :
        teams2_match[i] = 1 
    else: 
        teams2_match[i] = 0
english['direct_match_for_teams2'] = teams2_match

In [0]:
leagues_match = np.zeros(english.shape[0])
for i, leagues in enumerate(english['league_name_in_english']):
    if leagues in english['result.tsv'].values[i] :
        leagues_match[i] = 1 
    else: 
        leagues_match[i] = 0
english['direct_match_for_leagues'] = leagues_match

In [0]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [word for word in words if word and word not in stops]

    return words

In [0]:
%%time 
english['content_norm'] = english['result.tsv'].apply(normalize)

CPU times: user 10.2 s, sys: 93.1 ms, total: 10.3 s
Wall time: 10.3 s


In [0]:
english.head(3)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,team_of_season_1_name_in_english,team_of_season_2_name_in_english,league_name_in_english,direct_match_for_teams1,direct_match_for_teams2,direct_match_for_leagues,content_norm
0,Middlesbrough have woken to good times in thei...,7,0-0,bet365,2019-04-19 14:00:00,Чемпионшип,Мидлсбро,Сток Сити,Middlesbrough,Stoke City,Championship,1.0,1.0,0.0,"[middlesbrough, have, woken, to, good, times, ..."
1,Wolves prefer to attack with pace on the count...,7.5,0-0,bet365,2019-04-20 14:00:00,Премьер-лига,Вулверхэмптон,Брайтон,Wolverhampton,Brighton,Premier League,0.0,1.0,0.0,"[wolves, prefer, to, attack, with, pace, on, t..."
2,In my opinion a 0-0 draw is quite likely betwe...,13,0-0,bet365,2019-04-18 19:00:00,Лига Европы УЕФА,Валенсия,Вильярреал,Valencia,Villarreal,UEFA Europa League,1.0,1.0,0.0,"[in, my, opinion, a, 0-0, draw, is, quite, lik..."


In [0]:
w2v_model = gensim.models.Word2Vec([text for text in tqdm(english['content_norm'])], size = 300, workers=cpu_count())

100%|██████████| 89277/89277 [00:00<00:00, 1899012.49it/s]


In [0]:
w2v_model.save('w2v_sport_model.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
fasttxt_model = gensim.models.FastText([text for text in tqdm(english['content_norm'])], size = 300, workers=cpu_count())

100%|██████████| 89277/89277 [00:00<00:00, 2157768.78it/s]


In [0]:
fasttxt_model.save('fasttext_sport_model.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
from nltk import word_tokenize

In [0]:
for i, text in enumerate(english['content_norm'][10:20].values):
  # print(i, text)
    for word in text:
    # print(word)
        word2 = english['team_of_season_2_name_in_english'][i]
        word2 = word2.lower()
        if fasttxt_model.similarity(word, word2) > 0.4:
            print(word, word2)
            print(fasttxt_model.similarity(word, word2))

charlton stoke city
0.7328718
gillingham's stoke city
0.5907261
them stoke city
0.42481327
heavily stoke city
0.5012254
however stoke city
0.42537108
charlton stoke city
0.7328718
stick stoke city
0.43608606
bowyer stoke city
0.5250617
site stoke city
0.4507327
he stoke city
0.40542156
season stoke city
0.40246022
flourish stoke city
0.44016692
he stoke city
0.40542156
charlton stoke city
0.7328718
fluent stoke city
0.5366877
feisty stoke city
0.42089295
though stoke city
0.4263256
charlton stoke city
0.7328718
sound stoke city
0.5120495
bizarre stoke city
0.5059941
they stoke city
0.51146024
seasonal stoke city
0.4200187
charlton stoke city
0.7328718
milan villarreal
0.40570813
they villarreal
0.44555345
part villarreal
0.44959363
torino villarreal
0.5679932
according villarreal
0.4204432
barcelona villarreal
0.6598184
massive villarreal
0.48637205
liverpool villarreal
0.548046
they villarreal
0.44555345
resting villarreal
0.40855747
they villarreal
0.44555345
there villarreal
0.48307

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  import sys
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  if __name__ == '__main__':


In [0]:
print(fasttxt_model.most_similar('turkey'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[("turkey's", 0.804187536239624), ('crotia', 0.786919116973877), ('croatia', 0.7821520566940308), ('croitia', 0.7766404151916504), ('turks', 0.7752366065979004), ('cracovia', 0.7720080614089966), ('moldavia', 0.7700682878494263), ('gracia', 0.764377236366272), ('barotsia', 0.7630856037139893), ('benfica', 0.7554163336753845)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# FastText

### Команда 1 

In [0]:
model2 = 'fasttext_sport_model.model'
model_fast = gensim.models.FastText.load(model2)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
teams1_vector_match = np.zeros(english.shape[0])

In [0]:
for i, text in enumerate(english['content_norm'].values):
    for word in text:
        try:
            word2 = english['team_of_season_1_name_in_english'][i]
            word2 = word2.lower()
            #print(word2)
            if model_fast.similarity(word, word2) > 0.7:
                teams1_vector_match[i] = 1
        except:
            continue

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  import sys
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
english['ft_vector_match_team1'] = teams1_vector_match

In [0]:
all_vals = english['ft_vector_match_team1'].shape[0]
print(all_vals)

89277


In [0]:
success = english[english['ft_vector_match_team1'] == 1].shape[0]
print(success)

69673


In [0]:
vectorline_team1 = success / all_vals
print(vectorline_team1)

0.7804137683837943


### Команда 2 

In [0]:
teams2_vector_match = np.zeros(english.shape[0])

In [0]:
for i, text in enumerate(english['content_norm'].values):
    for word in text:
        try:
            word2 = english['team_of_season_2_name_in_english'][i]
            word2 = word2.lower()
            if model_fast.similarity(word, word2) > 0.7:
                teams2_vector_match[i] = 1
        except:
            continue

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
english['ft_vector_match_team2'] = teams2_vector_match

In [0]:
all_vals2 = english['ft_vector_match_team2'].shape[0]
print(all_vals2)

89277


In [0]:
success2 = english[english['ft_vector_match_team2'] == 1].shape[0]
print(success2)

67673


In [0]:
vectorline_team2 = success2 / all_vals2
print(vectorline_team2)

0.7580115819303964


### Лига

In [0]:
league_vector_match = np.zeros(english.shape[0])

In [0]:
for i, text in enumerate(english['content_norm'].values):
    for word in text:
        try:
            word2 = english['league_name_in_english'][i]
            word2 = word2.lower()
            if model_fast.similarity(word, word2) > 0.7:
                league_vector_match[i] = 1
        except:
            continue

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
english['ft_vector_match_league'] = league_vector_match

In [0]:
all_vals3 = english['ft_vector_match_league'].shape[0]
print(all_vals3)

89277


In [0]:
success3 = english[english['ft_vector_match_league'] == 1].shape[0]
print(success3)

16413


In [0]:
vectorline_league = success3 / all_vals3
print(vectorline_league)

0.18384354312980947


In [0]:
english.to_csv(r'english_vectormodels.csv', index = False)

# W2V

In [0]:
model1 = 'w2v_sport_model.model'
model_w2v = gensim.models.Word2Vec.load(model1)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
teams1_vector_match_w2v = np.zeros(english.shape[0])

In [0]:
for i, text in enumerate(english['content_norm'][:50].values):
    for word in text:
        word2 = english['team_of_season_1_name_in_english'][i]
        word2 = word2.lower()
        try:
            if model_w2v.similarity(word, word2) > 0.5:
                print(word, word2)
                print(model_w2v.similarity(word, word2))
        except:
            continue

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


middlesbrough middlesbrough
1.0
hull middlesbrough
0.60770744
city middlesbrough
0.501398
bolton middlesbrough
0.6349702
stoke middlesbrough
0.6533223
city middlesbrough
0.501398
wolves wolverhampton
0.5792721
brighton wolverhampton
0.5874122
brighton wolverhampton
0.5874122
newcastle wolverhampton
0.6015073
liverpool wolverhampton
0.56330746
brighton wolverhampton
0.5874122
brighton wolverhampton
0.5874122
wolves wolverhampton
0.5792721
brighton wolverhampton
0.5874122
villarreal valencia
0.6649103
valencia valencia
0.99999994
valencia valencia
0.99999994
charlton gillingham
0.65266067
charlton gillingham
0.65266067
charlton gillingham
0.65266067
charlton gillingham
0.65266067
charlton gillingham
0.65266067
they torino
0.52523226
torino torino
1.0
barcelona barcelona
0.99999994
liverpool barcelona
0.7332718
levante barcelona
0.62007606
duisburg duisburg
1.0
espanyol betis
0.5771512
betis betis
0.99999994
sassuolo fiorentina
0.6581077
fiorentina fiorentina
0.99999994
lorient lorient
1.

In [0]:
for i, text in enumerate(english['content_norm'].values):
    for word in text:
        try:
            word2 = english['team_of_season_1_name_in_english'][i]
            word2 = word2.lower()
            #print(word2)
            if model_w2v.similarity(word, word2) > 0.4:
                teams1_vector_match_w2v[i] = 1
                print(word, word2)
        except:
            continue

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  import sys
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Streaming output truncated to the last 5000 lines.
watford wolverhampton
city wolverhampton
city wolverhampton
norwich wolverhampton
arsenal wolverhampton
wolves wolverhampton
valencia wolverhampton
watford wolverhampton
wolves wolverhampton
watford wolverhampton
wolves wolverhampton
palace wolverhampton
watford wolverhampton
spurs wolverhampton
wolves wolverhampton
city wolverhampton
boavista boavista
primeira boavista
cd boavista
primeira boavista
boavista boavista
tondela boavista
boavista boavista
fulham fulham
and fulham
wigan fulham
lincoln blackpool
they blackpool
blackpool blackpool
holstein greuther
kiel greuther
greuther greuther
furth greuther
fulham fulham
and fulham
they fulham
wigan fulham
and fulham
and fulham
wigan fulham
blackpool blackpool
bournemouth bournemouth
west bournemouth
ham bournemouth
they bournemouth
spal juventus
juventus juventus
wigan fulham
they fulham
fulham fulham
fulham fulham
they fulham
and fulham
wigan fulham
and fulham
they fulham
and fulham
ful

In [0]:
english['w2v_vector_match_team1'] = teams1_vector_match_w2v

In [0]:
all_vals4 = english['w2v_vector_match_team1'].shape[0]
print(all_vals4)
success4 = english[english['w2v_vector_match_team1'] == 1].shape[0]
print(success4)
vectorline_league = success4 / all_vals4
print(vectorline_league)

89277
43253
0.48448088533440864


In [0]:
teams2_vector_match_w2v = np.zeros(english.shape[0])

In [0]:
for i, text in enumerate(english['content_norm'].values):
    for word in text:
        try:
            word2 = english['team_of_season_2_name_in_english'][i]
            word2 = word2.lower()
            #print(word2)
            if model_w2v.similarity(word, word2) > 0.7:
                teams2_vector_match_w2v[i] = 1
        except:
            continue

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  import sys
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
english['w2v_vector_match_team2'] = teams2_vector_match_w2v

In [0]:
all_vals5 = english['w2v_vector_match_team2'].shape[0]
print(all_vals5)
success5 = english[english['w2v_vector_match_team2'] == 1].shape[0]
print(success5)
vectorline_league = success5 / all_vals5
print(vectorline_league)

89277
35825
0.401279164846489


In [0]:
league_vector_match_w2v = np.zeros(english.shape[0])

In [0]:
for i, text in enumerate(english['content_norm'].values):
    for word in text:
        try:
            word2 = english['league_name_in_english'][i]
            word2 = word2.lower()
            if model_w2v.similarity(word, word2) > 0.7:
                league_vector_match_w2v[i] = 1
        except:
            continue

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
english['w2v_vector_match_league'] = league_vector_match_w2v

In [0]:
all_vals6 = english['w2v_vector_match_league'].shape[0]
print(all_vals6)
success6 = english[english['w2v_vector_match_league'] == 1].shape[0]
print(success6)
vectorline_league = success6 / all_vals6
print(vectorline_league)

89277
2010
0.02251419738566484
